# One run full walktrhough 

* Do the full walk through on the large data set
* Refactor the source code and bring it to individual scripts
* Ensure a full run with one click

In [2]:
## check some parameters
## depending where you launch your notebook, the relative path might not work
## you should start the notebook server from your base path
## when opening the notebook, typically your path will be ../ads_covid-19/notebooks
import os
if os.path.split(os.getcwd())[-1]=='notebooks':
    os.chdir("../")

'Your base path is at: '+os.path.split(os.getcwd())[-1]

'Your base path is at: ads_covid-19'

## 1 Update all data

In [3]:
import subprocess
import os

import pandas as pd
import numpy as np

from datetime import datetime

import requests
import json

def get_johns_hopkins():
    ''' Get data by a git pull request, the source code has to be pulled first
        Result is stored in the predifined csv structure
    '''
    git_pull = subprocess.Popen( "/usr/bin/git pull" ,
                         cwd = os.path.dirname( 'data/raw/COVID-19/' ),
                         shell = True,
                         stdout = subprocess.PIPE,
                         stderr = subprocess.PIPE )
    (out, error) = git_pull.communicate()


    print("Error : " + str(error))
    print("out : " + str(out))



if __name__ == '__main__':
    get_johns_hopkins()
   


Error : b'The system cannot find the path specified.\r\n'
out : b''


## 2. Process pipeline 

In [4]:
import pandas as pd
import numpy as np

from datetime import datetime


def store_relational_JH_data():
    ''' Transformes the COVID data in a relational data set

    '''

    data_path='data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
    df_code = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
    pd_raw=pd.read_csv(data_path)

    
    time_idx = pd_raw.columns[4:]
    df_plot = pd.DataFrame({
        'date':time_idx})
    df_input_large= pd_raw['Country/Region'].unique()
    for each in df_input_large:
        df_plot[each] =np.array(pd_raw[pd_raw['Country/Region']==each].iloc[:,4::].sum(axis=0))
    df = df_plot.drop('date', axis=1)
    
    #Merging the data set over COUNTRY for CODE column
    world_raw =  pd.DataFrame({"COUNTRY" : df_input_large, "Confirm cases" :df.iloc[-1]})
    world_con = pd.merge(world_raw, df_code, on = "COUNTRY").drop('GDP (BILLIONS)', axis=1)
    world_con.to_csv('data/processed/COVID_CRD.csv',sep=';',index=False)
    
    pd_data_base=pd_raw.rename(columns={'Country/Region':'COUNTRY',
                      'Province/State':'state'})

    pd_data_base['state']=pd_data_base['state'].fillna('no')

    pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


    pd_relational_model_1=pd_data_base.set_index(['state','COUNTRY']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
    pd_relational_model = pd.merge(pd_relational_model_1, df_code, on = "COUNTRY").drop('GDP (BILLIONS)', axis=1)
    pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')
    
    pd_relational_model.to_csv('data/processed/20200823_COVID_relational_confirmed.csv',sep=';',index=False)
    
    print(df_input_large)
    print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
    print(' Latest date is: '+str(max(pd_relational_model.date)))
if __name__ == '__main__':

    store_relational_JH_data()


['Afghanistan' 'Albania' 'Algeria' 'Andorra' 'Angola'
 'Antigua and Barbuda' 'Argentina' 'Armenia' 'Australia' 'Austria'
 'Azerbaijan' 'Bahamas' 'Bahrain' 'Bangladesh' 'Barbados' 'Belarus'
 'Belgium' 'Belize' 'Benin' 'Bhutan' 'Bolivia' 'Bosnia and Herzegovina'
 'Botswana' 'Brazil' 'Brunei' 'Bulgaria' 'Burkina Faso' 'Burma' 'Burundi'
 'Cabo Verde' 'Cambodia' 'Cameroon' 'Canada' 'Central African Republic'
 'Chad' 'Chile' 'China' 'Colombia' 'Comoros' 'Congo (Brazzaville)'
 'Congo (Kinshasa)' 'Costa Rica' "Cote d'Ivoire" 'Croatia' 'Cuba' 'Cyprus'
 'Czechia' 'Denmark' 'Diamond Princess' 'Djibouti' 'Dominica'
 'Dominican Republic' 'Ecuador' 'Egypt' 'El Salvador' 'Equatorial Guinea'
 'Eritrea' 'Estonia' 'Eswatini' 'Ethiopia' 'Fiji' 'Finland' 'France'
 'Gabon' 'Gambia' 'Georgia' 'Germany' 'Ghana' 'Greece' 'Grenada'
 'Guatemala' 'Guinea' 'Guinea-Bissau' 'Guyana' 'Haiti' 'Holy See'
 'Honduras' 'Hungary' 'Iceland' 'India' 'Indonesia' 'Iran' 'Iraq'
 'Ireland' 'Israel' 'Italy' 'Jamaica' 'Japan' 'Jo

## 3  Filter and Doubling Rate Calculation

In [5]:
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)
import pandas as pd

from scipy import signal


def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)



    return result




def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','COUNTRY',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','COUNTRY',filter_on]].groupby(['state','COUNTRY']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()





def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','COUNTRY',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','COUNTRY']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


if __name__ == '__main__':
    test_data_reg=np.array([2,4,6])
    result=get_doubling_time_via_regression(test_data_reg)
    print('the test slope is: '+str(result))

    pd_JH_data=pd.read_csv('data/processed/COVID_relational_confirmed.csv',sep=';',parse_dates=[0])
    pd_JH_data=pd_JH_data.sort_values('date',ascending=True).copy()

    #test_structure=pd_JH_data[((pd_JH_data['country']=='US')|
    #                  (pd_JH_data['country']=='Germany'))]

    pd_result_larg=calc_filtered_data(pd_JH_data)
    pd_result_larg=calc_doubling_rate(pd_result_larg)
    pd_result_larg=calc_doubling_rate(pd_result_larg,'confirmed_filtered')


    mask=pd_result_larg['confirmed']>100
    pd_result_larg['confirmed_filtered_DR']=pd_result_larg['confirmed_filtered_DR'].where(mask, other=np.NaN)
    pd_result_larg.to_csv('data/processed/COVID_final_set.csv',sep=';',index=False)
    print(pd_result_larg[pd_result_larg['COUNTRY']=='Germany'].tail())


the test slope is: [2.]
            date state  COUNTRY  confirmed  confirmed_filtered  confirmed_DR  \
31025 2020-08-18    no  Germany     228120            228165.0    145.588821   
31026 2020-08-19    no  Germany     229706            229769.4    151.813262   
31027 2020-08-20    no  Germany     231292            231201.6    144.833544   
31028 2020-08-21    no  Germany     233029            232682.1    139.237035   
31029 2020-08-22    no  Germany     233861            234162.6    181.181264   

       confirmed_filtered_DR  
31025             178.365804  
31026             153.949953  
31027             151.295528  
31028             158.765201  
31029             157.164539  


In [6]:
#pd_result_larg[pd_result_larg['COUNTRY']=='US']

In [7]:
#print(pd_result_larg[pd_result_larg['COUNTRY']=='Germany'].tail())

## 4 Visual Board

In [10]:
import pandas as pd
import numpy as np
import dash_bootstrap_components as dbc

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State


import plotly.graph_objects as go
from plotly import tools

import os
print(os.getcwd())
df_input_large=pd.read_csv('data/processed/COVID_final_set.csv',sep=';')
world_con=pd.read_csv('data/processed/COVID_CRD.csv',sep=';')
df_input_sir = pd.read_csv('data/processed/COVID_sir_fitted_table.csv',sep=';')
df_all = df_input_sir.columns
df_all = list(df_all)

fig = go.Figure()
fig_2 = go.Figure()
fig_3 =  go.Figure(data=go.Choropleth(
                locations = world_con['CODE'],
                z = world_con['Confirm cases'],
                text = world_con['COUNTRY'],
                colorscale = 'Blues',
                autocolorscale=False,
                reversescale=False,
                marker_line_color='darkgray',
                marker_line_width=0.5,
                colorbar_title = 'Confirmed cases',
                ))
fig_3.update_layout(title_text='COVID 19 WORLD MAP',
                    width=1700,
                    height=720,
                    geo=dict(
                        showframe=False,
                        showcoastlines=False,
                        projection_type='equirectangular'
                    ),
                    annotations = [dict(
                        x=0.55,
                        y=0.1,
                        xref='paper',
                        yref='paper',
                        text='World Map',
                        showarrow = False
                    )]
                )


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
app.title = 'COVID-19 Dashboard'

app.layout = html.Div([
        
        dbc.Row(dbc.Col(html.H3('Enterprise Data Science COVID-19 Data Analytics'),
                        width={'size': 6, 'offset': 1},
                        ),
                ),
        
        dbc.Row(dbc.Col(html.Div('''
                            Goal of the project is to teach data science by applying a cross industry standard process,
                            it covers the full walkthrough of: automated data gathering, data transformations,
                            filtering and machine learning to approximating the doubling time, and
                            (static) deployment of responsive dashboard. 
                            '''),
                        width={'size': 10, 'offset': 1},
                        )
                ),
        
        dbc.Row(
            [
                dbc.Col(dcc.Dropdown(
                            id='country_dropdown',
                            options=[ {'label': each,'value':each} for each in df_input_large['COUNTRY'].unique()],
                            value=['US', 'Germany','India'], # which are pre-selected
                            multi= True),
                        width={'size': 5, "offset": 0, 'order': 'first'}
                        ),
                dbc.Col(dcc.Dropdown(
                            id='country_dropdown_sir',
                            options=[ {'label': each,'value':each} for each in df_all[1:]],
                            value='India', # which are pre-selected
                            multi= False
                            ),
                        width={'size': 5, "offset": 2, 'order': 'second'}
                        ),
                ], no_gutters=True
        ),
    
        dbc.Row(
            [
                dbc.Col(
                        dcc.Dropdown(
                            id='doubling_time',
                            options=[
                                {'label': 'Timeline Confirmed ', 'value': 'confirmed'},
                                {'label': 'Timeline Confirmed Filtered', 'value': 'confirmed_filtered'},
                                {'label': 'Timeline Doubling Rate', 'value': 'confirmed_DR'},
                                {'label': 'Timeline Doubling Rate Filtered', 'value': 'confirmed_filtered_DR'}
                            ],
                            value='confirmed',
                            multi=False
                            ),
                        width={'size': 3, "offset": 0, 'order': 'first'}
                        ),
                
                
                ], 
        ),
                
        dbc.Row(
            [
                dbc.Col(dcc.Graph(
                            figure=fig, 
                            id='main_window_slope'
                            ),
                        width=6, md={'size': 5,  "offset": 0, 'order': 'first'}
                        ),
                
                dbc.Col(dcc.Graph(
                            id='SIR_model'
                            ),
                        width=6, md={'size': 5,  "offset": 1, 'order': 'last'}
                        ),
            ]
        ),
        
         dbc.Row(
                dbc.Col(dcc.Graph(
                            figure=fig_3,
                            id='World_map'
                        ),
                        width=12, md={'size': 12,  "offset": 0, 'order': 'first'}
                        ),
         )


])

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_dropdown', 'value'),
    Input('doubling_time', 'value')])

def update_figure(country_list,show_doubling):


    if 'doubling_rate' in show_doubling:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:

        df_plot=df_input_large[df_input_large['COUNTRY']==each]

        if show_doubling=='doubling_rate_filtered':
            df_plot=df_plot[['state','COUNTRY','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['COUNTRY','date']).agg(np.mean).reset_index()
        else:
            df_plot=df_plot[['state','COUNTRY','confirmed','confirmed_filtered','confirmed_DR','confirmed_filtered_DR','date']].groupby(['COUNTRY','date']).agg(np.sum).reset_index()
       #print(show_doubling)


        traces.append(dict(x=df_plot.date,
                                y=df_plot[show_doubling],
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=800,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        ) 
    }

@app.callback(
    Output('SIR_model', 'figure'),
    [Input('country_dropdown_sir', 'value')])

def SIR_fig(con_input):
    df= df_input_sir
    
    for i in df[1:]:
        data = []
        trace = go.Scatter(x=df.date,
                        y=df[con_input],
                        mode='lines+markers',
                        name = con_input)
        data.append(trace)
        
        trace_fitted = go.Scatter(x=df.date,
                        y=df[con_input +'_fitted'], 
                        mode='lines+markers',
                        name=con_input+'_fitted')
        data.append(trace_fitted)
        
        
            
    return {'data': data,
            'layout' : dict(
                width=800,
                height=720,
                title= 'SIR model',
                xaxis={'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },
                yaxis={'type':"log",
                       'range':'[1.1,5.5]'
                      }
                
            )
        }

if __name__ == '__main__':

    app.run_server(debug=True, port= 8051, use_reloader=False)


C:\Users\Vishal Sharbidar\Desktop\ADS_COVID-19\ads_covid-19
Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

Dash is running on http://127.0.0.1:8051/

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.

 in production, use a production WSGI server like gunicorn instead.



 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
